In [ ]:
import requests
import pandas as pd

# URL of the REST service
TE_summary_url = "https://www.laketahoeinfo.org/WebServices/GetThresholdEvaluations/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476"

# Fetch the data from the REST service
response = requests.get(TE_summary_url)
response.raise_for_status()  # Raise an error for bad status codes

# Parse the JSON response and convert to DataFrame
data = response.json()
df = pd.DataFrame(data)

#  keep only the specified columns
columns_to_keep = ['ThresholdCategory', 'ThresholdReportingCategory', 'IndicatorName', 'Status2019', 'Status2023', 'Trend2019', 'Trend2023']

df = df[columns_to_keep]

# Print to an excel doc
#filepath = r"C:\Users\snewsome\Documents\Threshold\TE_summary.xlsx"
#df.to_excel(filepath, index=False)


In [6]:
#Process the data

#Calculate the number of thresholds that are improving, declining, or stable
df['Trend2019'] = df['Trend2019'].str.lower()
df['Trend2023'] = df['Trend2023'].str.lower()

# Define Trend Calls
trend_calls = {
    'rapid improvement': 1,
    'moderate improvement': 2,
    'little or no change': 3,
    'moderate decline': 4,
    'rapid decline': 5
}

# Map the trend calls to numerical values
df['Trend2019'] = df['Trend2019'].map(trend_calls)
df['Trend2023'] = df['Trend2023'].map(trend_calls)
# Calculate the trend change
df['Trend Change'] = df['Trend2023'] - df['Trend2019']

# Categorize the trend changes
df['Trend Category'] = df['Trend Change'].apply(lambda x: 'improving' if x < 0 else ('declining' if x > 0 else 'stable'))

# Calculate the number of thresholds in each category
trend_counts = df['Trend Category'].value_counts()

print(trend_counts)

#Define Status Calls
attainment_calls = {'Attainment':'Considerabley better than target', 'Attainment': 'At or somewhat better than target',
 'Non-Attainment':'Somewhat worse than target', 'Non-Attainment':'Considerably worse than target',
 'Not Applicable': 'Insufficient data or no target established', 'Implemented': 'Not Applicable'
 }

# Map the attainment calls based on Status
df['Attainment2019'] = df['Status2019'].map(attainment_calls)
df['Attainment2023'] = df['Status2023'].map(attainment_calls)

#calcluate change in attainment for 2023 vs 2019
df['Attainment Change'] = df['Attainment2023'] - df['Attainment2019']
# Calculate the number of thresholds in each category
attainment_counts = df['Attainment Change'].value_counts()
print (attainment_counts)
# Categorize the attainment changes
df['Attainment Category'] = df['Attainment Change'].apply(lambda x: 'improving' if x < 0 else ('declining' if x > 0 else 'stable'))

AttributeError: Can only use .str accessor with string values!